## Implementation of Gibbs Sampling to approximate probability queries from a Bayesian Network.

In [8]:
import random
import numpy as np
from calculations_helper import create_factors, join_factors, find_corresponding_rows

def estimate_gibbs(iterations: int, network: dict, queries: list[int], evidence: dict[int,bool]) -> np.array:
    """Generate an estimate for the probability distribution for a given set of query variables and evidence values

    Args:
        iterations (int): number of samples to take before we go with the estimate
        network (dict): underlying bayesian network
        queries (list[int]): list of query variables
        evidence (dict[int,bool]): list of evidence variables with their respective values

    Returns:
        np.array: estimated probability distribution for the different combinations the query variables can take on
    """
    prob_distribution = np.zeros(shape=1<<len(queries))
    
    # create list of non-evidence variables
    non_evidence_variables = [int(i) for i in network.keys() if int(i) not in evidence.keys()]

    # create a helper function to find the probability of a certain variable taking on a value given we know other variables
    def calculate_probability(var: int, others: dict[int,bool]) -> float:
        """Given a variable and set values for all other variables in the above Bayesian Network, calculate the the probability of var being true

        Args:
            var (int): variable in question
            others (dict[int,bool]): all other variable values

        Returns:
            bool: probability that var is true given all other variable values
        """
        # join on the factors that matter to var
        factor_idx_to_factor, var_to_factor_indices = create_factors(network=network, evidence=others) 
        # the following result should be a 2D vector - var is False or var is True
        var_distribution = join_factors(var, eliminate=False, relevant_factors=[factor_idx_to_factor[i] for i in var_to_factor_indices[var]])[1]
        var_distribution = var_distribution / np.sum(var_distribution)
        return var_distribution[1] # probability of var being true
    
    current_evidence = evidence
    # randomly initialize all the variables
    for v in non_evidence_variables:
        current_evidence[v] = (random.random() < 0.5)

    for _ in range(iterations):
        # perform the Gibbs algorithm this many times
        var_to_change = non_evidence_variables[int(random.random()*len(non_evidence_variables))]
        del current_evidence[var_to_change]
        # simulate the probability of var_to_change being true given everything else
        p = calculate_probability(var_to_change, current_evidence)
        current_evidence[var_to_change] = (random.random() < p)
        # see how our queries showed up
        query_values = [1 if current_evidence[q] else 0 for q in queries]
        # find the entry in our probability distribution that corresponds with this combination of query values
        posn = find_corresponding_rows(query_values, queries, queries)[0]
        # update the distribution accordingly
        prob_distribution[posn] += 1.0

    return prob_distribution / iterations # normalization into a probability

In [9]:
import json

queries = [0]
evidence = {3:True,4:True}

with open('bn_test_1.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(1000, bayesian_network, queries, evidence))

[0.696 0.304]


In [10]:
import json

queries = [0, 3]
evidence = {2:True}

with open('bn_test_2.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(1000, bayesian_network, queries, evidence))

[0.34  0.047 0.1   0.513]


In [11]:
import json

queries = [1]
evidence = {2:False}

with open('bn_test_3.json') as f:
    bayesian_network = json.load(f)
    print(estimate_gibbs(1000, bayesian_network, queries, evidence))

[0.842 0.158]
